# TPM034A Machine Learning for socio-technical systems 
## `Assignment 01: Discover, explore and visualise data`

**Delft University of Technology**<br>
**Q2 2022**<br>
**Instructor:** Sander van Cranenburgh <br>
**TAs:**  Francisco Garrido Valenzuela & Lucas Spierenburg <br>

### `Instructions`

**Assignments aim to:**<br>
* Examine your understanding of the key concepts and techniques.
* Examine your the applied ML skills.

**Assignments:**<br>
* Are graded and must be submitted (see the submission instruction below). 

### `Workspace set-up`
**Option 1: Google Colab**<br>
Uncomment the following cells code lines if you are running this notebook on Colab

In [ ]:
#!git clone https://github.com/TPM34A/Q2_2022
#!pip install -r Q2_2022/requirements_colab.txt
#!mv "/content/Q2_2022/Assignments/assignment_01/data" /content/data

**Option 2: Local environment**<br>
Uncomment the following cell if you are running this notebook on your local environment. This will install all dependencies on your Python version.

In [ ]:
#!pip install -r requirements.txt

## `Application: Liveability and affordable housing in Amsterdam` <br>

### **Introduction**
There is a widespread sense that affordable housing for the middle incomes households is under pressure. Especially for new entrants to the housing markets (i.e. those who do not yet own a house), affordable houses to buy in pleasant neighboorhoods are in short supply. Entrants to the housing market typically are people in their 20s and 30s.<br>

The municipality of Amsterdam would like to tackle this issue. (see https://openresearch.amsterdam/en/page/77950/housing-crisis for articles on the subject). However, at present, the municipality of Amsterdam lacks insights on the extent to which access to affordable houses has deteriorated. <br>

*Your are asked to assist the municipality of Amsterdam in investigating **whether** and **where** access to afforable houses has deteriorated.*<br>

### **Data**

You have access to four data sets:
1. Real-estate prices in Amsterdam, at buurt level
1. Liveability scores in the Netherlands, at buurt level
1. Population statistics in the Netherlands, at buurt level
1. Buurten boundaries in the Netherlands (GIS)

### **Notes**
- In the livability scores dataset the column *versie* show the different versions of the livability score, only use the 3rd version. Thus, you may filter this column to keep *Leefbaarometer 3.0*	only.
- You may assume that the population statistics and geospatial data have not substantially changed across the years 2014 and 2020. Thus, you may assume both apply to 2014 and 2020.
- For Population statistics (3rd dataset), [this document](data/buurt/metadata_buurt.csv) provides a brief explanation of the features.

### **Tasks and grading**

Your assigment is divided into 3 subtasks: (1) Data preparation, (2) Data exploration and (3) Assess the affordability of liveable neighborhoods. In total, 10 points can be earned in this assignment. The weight per subtask is shown below. 

1.  **Data preparation: construct data from multiple data sources.** [2 pnt]
    1. Load the four dataset and show a preview of the dataset structure (some DataFrame rows).
    1. Prepare the table data (non-GIS) to have two different DataFrames (for 2014 and 2020) that contains the following information:
        - the liveability data for the year of interest, using the 3rd version of the Leefbaarometer
        - population data 
        - Real-estate prices
        - at the buurt level
        - *Make sure to filter the data and remove NULL (NaN values) if required*
    1. Add the geographic component of the buurten to your data.
1. **Data exploration: discover and visualise data.** [4 pnt]
    1.  Investigate the statistical distribution of the real-estate price levels and liveability in both years, using either a histogram of a CDF.
    1. Visualise the correlation between real-estate prices and liveability in Amsterdam, at the buurt level, with a scatter plot for each year. Then, visualise spatially real-estate prices and liveability in Amsterdam for 2014 and 2020 (use the same color scale for years 2014 and 2020).
1. **Assess the change in affordability of liveable neighbourhoods.** [3 pnt]
    1. Explore how the change in liveability associates with a change in real-estate prices, using a scatter plot.
    1. Compute the ratio of the liveability score over the real-estate price for both years, and show how the distribution of the ratio of liveability over real-estate price has changed between the two years.
    1. Determine the 5 buurten in which the ratio of liveability over real-estate has deteriorated most.
    1. Determine whether the number of buurten with price < 5k euro/m2 and a liveability ratio > 1/k euro has decreased in 2020, compared to 2014
1. **Qualitative reflection on machine learning and generalisation: There are some buurts in Amsterdam for which real-estate price data were missing. Suppose the minicipality of Amsterdam asks you whether you can create a machine learning model that can predict real-estate prices from the liveability index. Do you think this is possible? Explain your answer (conceptually).** [1 pnt]

### **Submission**
- The deadline for this assignment is **Wed, 23 November 2022** 
- Use **Python 3.7 or above**
- You have to submit your work in zip file with the ipynb (fully executed) in brightspace

In [ ]:
import os
from os import getcwd
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.api import OLS, add_constant, tools
pd.set_option('display.max_columns', None)

### 1. Data preparation: construct data from multiple data sources [2 pnt]
#### 1.1 Load the four dataset and show a preview of the dataset structure (some DataFrame rows).

##### 1.1.1 Load datasets

In [ ]:
# Set the data folder path
data_folder = Path(f'data')

In [ ]:
# Load three datasets as pandas dataframes
buurt_price_data = pd.read_csv(data_folder/'real_estate_price/buurt_price.csv')
buurt_liv_data = pd.read_csv(data_folder/'liveability/buurt_liveability.csv')
buurt_pop_data = pd.read_csv(data_folder/'buurt/buurt_2020_pop.csv')

# Load the shapefile as geopandas object
buurt_shape = gpd.read_file(data_folder/'buurt/buurt 2020.shp')

In [ ]:
buurt_price_data

In [ ]:
buurt_liv_data

In [ ]:
buurt_pop_data

In [ ]:
buurt_shape

#### 1.2 Prepare the table data (non-GIS) to have two different DataFrames (for 2014 and 2020) that contains the following information:
- the liveability data for the year of interest, using the 3rd version of the Leefbaarometer
- population data 
- Real-estate prices
- at the buurt level
- *Make sure to filter the data and remove NULL (NaN values) if required*

##### 1.2.1 Liveability DataFrame at buurt level

In [ ]:
# Select only the necessary columns
buurt_liv_data = buurt_liv_data[['bu_code', 'versie', 'jaar', 'bu_naam', 'lbm']]
buurt_liv_data

In [ ]:
def clean_liv_data(liv_data, year):
    """Select the right version and year for the liveability data"""

    # Extract subsets: only for the given year and Leefbaarometer 3.0 data
    liv_data_extracted = liv_data.loc[(liv_data.versie ==  'Leefbaarometer 3.0') & (liv_data.jaar == year)]

    # Return number of NaN values in the lbm column before removal
    print('The number of NaN lbm values in the', year, 'liveability dataset before removal is', liv_data_extracted['lbm'].isna().sum())

    # Remove rows where lbm is NaN
    liv_data_extracted_cleaned = liv_data_extracted.dropna(subset = ['lbm'])

    return liv_data_extracted_cleaned


buurt_liv_data_2014 = clean_liv_data(buurt_liv_data, 2014)
buurt_liv_data_2020 = clean_liv_data(buurt_liv_data, 2020)

In [ ]:
buurt_liv_data_2014

In [ ]:
buurt_liv_data_2020

##### 1.2.2 Population DataFrame at buurt level

In [ ]:
# Rename column to enable the same column name as key in all DataFrames
buurt_pop_data.rename(columns={"BU_CODE": "bu_code"}, inplace=True)

In [ ]:
buurt_pop_data

##### 1.2.3 Real estate price DataFrame at buurt level

In [ ]:
# Drop unnecessary real estate price column
buurt_price_data_2014 = buurt_price_data.drop('price_2020', axis=1)
buurt_price_data_2020 = buurt_price_data.drop('price_2014', axis=1)

In [ ]:
buurt_price_data_2014

In [ ]:
buurt_price_data_2020

##### 1.2.4 Combining DataFrames

In [ ]:
# Import reduce to enable multi-DataFrame merge
from functools import reduce

In [ ]:
# Select the DataFrames to be merged for the 2014 DataFrame
dfs = [buurt_price_data_2014, buurt_liv_data_2014, buurt_pop_data]

buurt_data_2014 = reduce(lambda  left,right: pd.merge(left,right,on=['bu_code'],
                                                      how='inner'), dfs)

buurt_data_2014

In [ ]:
# Select the DataFrames to be merged for the 2020 DataFrame
dfs = [buurt_price_data_2020, buurt_liv_data_2020, buurt_pop_data]

buurt_data_2020 = reduce(lambda  left,right: pd.merge(left,right,on=['bu_code'], how='inner'), dfs)

buurt_data_2020

#### 1.3 Add the geographic component of the buurten to your data

##### 1.3.1 Merge DataFrame with GeoDataFrame

In [ ]:
# Merge the DataFrame for both years with the Geopandas Shapefile
buurt_df_2014 = buurt_shape.merge(buurt_data_2014, on="bu_code")
buurt_df_2020 = buurt_shape.merge(buurt_data_2020, on="bu_code")

In [ ]:
buurt_df_2014

In [ ]:
buurt_df_2020

### 2. Data exploration: discover and visualise data [4 pnt]
#### 2.1 Investigate the statistical distribution of the real-estate price levels and liveability in both years

##### 2.1.1 Real estate price levels

In [ ]:
# Create histogram and empirical CDF for the real estate price levels of both 2014 and 2020
fig, axes = plt.subplots(1, 4, figsize=(15, 5), sharex=False)

sns.histplot(ax = axes[0],x = buurt_df_2014.price_2014)
sns.ecdfplot(ax = axes[1],x = buurt_df_2014.price_2014)
sns.histplot(ax = axes[2],x = buurt_df_2020.price_2020)
sns.ecdfplot(ax = axes[3],x = buurt_df_2020.price_2020)

axes[0].set_xlabel("real estate price 2014 (euro/m^2)")
axes[1].set_xlabel("real estate price 2014 (euro/m^2)")
axes[2].set_xlabel("real estate price 2020 (euro/m^2)")
axes[3].set_xlabel("real estate price 2020 (euro/m^2)")

axes[1].grid(True,linewidth = 0.5)
axes[1].minorticks_on()
axes[1].grid(which='minor', linestyle='dotted', linewidth='0.5', color='black')
axes[3].grid(True,linewidth = 0.5)
axes[3].minorticks_on()
axes[3].grid(which='minor', linestyle='dotted', linewidth='0.5', color='black')

##### 2.1.2 Liveability

In [ ]:
# Create histogram and empirical CDF for the liveability of both 2014 and 2020
fig, axes = plt.subplots(1, 4, figsize=(15, 5), sharex=False)

sns.histplot(ax = axes[0],x = buurt_df_2014.lbm)
sns.ecdfplot(ax = axes[1],x = buurt_df_2014.lbm)
sns.histplot(ax = axes[2],x = buurt_df_2020.lbm)
sns.ecdfplot(ax = axes[3],x = buurt_df_2020.lbm)

axes[0].set_xlabel("liveability 2014")
axes[1].set_xlabel("liveability 2014")
axes[2].set_xlabel("liveability 2020")
axes[3].set_xlabel("liveability 2020")

axes[1].grid(True,linewidth = 0.5)
axes[1].minorticks_on()
axes[1].grid(which='minor', linestyle='dotted', linewidth='0.5', color='black')
axes[3].grid(True,linewidth = 0.5)
axes[3].minorticks_on()
axes[3].grid(which='minor', linestyle='dotted', linewidth='0.5', color='black')

#### 2.2 Visualise the correlation between real-estate prices and liveability in Amsterdam, at the buurt level, with a scatter plot for each year. Then, visualise spatially real-estate prices and liveability in Amsterdam for 2014 and 2020 (use the same color scale for years 2014 and 2020).

##### 2.2.1 Scatterplots

In [ ]:
# Create scatterplots without and with regression lines for real estate prices and liveability
fig, axes = plt.subplots(2, 2, figsize=(20, 8), sharex=True, sharey=True)
fig.set_tight_layout(True)

sns.scatterplot(ax = axes[0,0], x = buurt_df_2014.price_2014, y = buurt_df_2014.lbm)
sns.scatterplot(ax = axes[0,1], x = buurt_df_2020.price_2020, y = buurt_df_2020.lbm)

sns.regplot(ax= axes[1,0], x = buurt_df_2014.price_2014, y = buurt_df_2014.lbm)
sns.regplot(ax= axes[1,1], x = buurt_df_2020.price_2020, y = buurt_df_2020.lbm)

axes[0,0].set_xlabel("real estate price 2014 (euro/m^2)")
axes[0,1].set_xlabel("real estate price 2020 (euro/m^2)")
axes[1,0].set_xlabel("real estate price 2014 (euro/m^2)")
axes[1,1].set_xlabel("real estate price 2020 (euro/m^2)")

plt.show()

##### 2.2.2 Maps

In [ ]:
# Create spatial maps showing the buurten of Amsterdam with real estate prices and liveability levels for both 2014 and 2020
fig, axes = plt.subplots(2, 2, figsize=(25, 25), sharex=True, sharey=True)
fig.set_tight_layout(True)

buurt_df_2014.plot(ax=axes[0,0], column = 'price_2014', legend=True, vmin=1.5, vmax=10)
buurt_df_2020.plot(ax=axes[0,1], column = 'price_2020', legend=True, vmin=1.5, vmax=10)

buurt_df_2014.plot(ax=axes[1,0], column = 'lbm', legend=True)
buurt_df_2020.plot(ax=axes[1,1], column = 'lbm', legend=True)

axes[0,0].set_title("real estate price 2014 (euro/m^2)")
axes[0,0].axis('off')
axes[0,1].set_title("real estate price 2020 (euro/m^2)")
axes[0,1].axis('off')
axes[1,0].set_title("liveability 2014")
axes[1,0].axis('off')
axes[1,1].set_title("liveability 2020")
axes[1,1].axis('off')

plt.show()

### 3. Assess the change in affordability of liveable neighbourhoods [3 pnt]

#### 3.1 Explore how the change in liveability associates with a change in real-estate prices, using a scatter plot.

In [ ]:
# Rename columns to make liveability score more easily comparable
buurt_df_2014_renamed  = buurt_df_2014.rename(columns={"lbm": "lbm_2014"})
buurt_df_2020_renamed = buurt_df_2020.rename(columns={"lbm": "lbm_2020"})

In [ ]:
# Merge both DataFrames to create a more comprehensible one
buurt_df = buurt_df_2014_renamed.merge(buurt_df_2020_renamed, on='bu_code')

In [ ]:
buurt_df

In [ ]:
# Calculate the change for both values
buurt_df['lbm_change'] = buurt_df['lbm_2020'] - buurt_df['lbm_2014']
buurt_df['price_change'] = buurt_df['price_2020'] - buurt_df['price_2014']

In [ ]:
# Create scatterplot without and with regression line for real estate price change and liveability change
fig, axes = plt.subplots(1, 2, figsize=(20, 8), sharex=True, sharey=True)
fig.set_tight_layout(True)

sns.scatterplot(ax = axes[0], x = buurt_df.price_change, y = buurt_df.lbm_change)
sns.regplot(ax= axes[1], x = buurt_df.price_change, y = buurt_df.lbm_change)

axes[0].set_xlabel("price change from 2014 to 2020")
axes[1].set_xlabel("price change from 2014 to 2020")

plt.show()

#### 3.2 Compute the ratio of the liveability score over the real-estate price for both years, and show how the distribution of the ratio of liveability over real-estate price has changed between the two years.

In [ ]:
# Calculate ratio for both 2014 and 2020
buurt_df['ratio_2014'] = buurt_df['lbm_2014'] / buurt_df['price_2014']
buurt_df['ratio_2020'] = buurt_df['lbm_2020'] / buurt_df['price_2020']

In [ ]:
# Create histogram and empirical CDF for the real estate price levels of both 2014 and 2020
fig, axes = plt.subplots(1, 4, figsize=(15, 5), sharex=False)

sns.histplot(ax = axes[0],x = buurt_df.ratio_2014)
sns.ecdfplot(ax = axes[1],x = buurt_df.ratio_2014)
sns.histplot(ax = axes[2],x = buurt_df.ratio_2020)
sns.ecdfplot(ax = axes[3],x = buurt_df.ratio_2020)

axes[0].set_xlabel("ratio 2014")
axes[1].set_xlabel("ratio 2014")
axes[2].set_xlabel("ratio 2020")
axes[3].set_xlabel("ratio 2020")

axes[1].grid(True,linewidth = 0.5)
axes[1].minorticks_on()
axes[1].grid(which='minor', linestyle='dotted', linewidth='0.5', color='black')
axes[3].grid(True,linewidth = 0.5)
axes[3].minorticks_on()
axes[3].grid(which='minor', linestyle='dotted', linewidth='0.5', color='black')

#### 3.3 Determine the 5 buurten in which the ratio of liveability over real-estate has deteriorated most.

In [ ]:
buurt_df['ratio_change'] = buurt_df['ratio_2020'] - buurt_df['ratio_2014']
buurt_df.nsmallest(5, 'ratio_change').bu_naam_x_x

#### 3.4 Determine whether the number of buurten with price < 5k euro/m2 and a liveability ratio > 1/k euro has decreased in 2020, compared to 2014

In [ ]:
print('Number in 2014:', len(buurt_df.loc[(buurt_df.price_2014 < 5) & (buurt_df.ratio_2014 > 1)]))
print('Number in 2020:', len(buurt_df.loc[(buurt_df.price_2020 < 5) & (buurt_df.ratio_2020 > 1)]))

So the number definitely has decreased significantly

### 4. Qualitative reflection on machine learning and generalisation [1 pnt]

I do think it is possible to predict real estate prices using the liveability index as input. As we can see in the scatterplots under 2.2.1 Scatterplots, the real estate prices do seem to correlate quitte well with the liveability index. In addition, there also seems to be a relation between the change in real estate price level and the change in liveability index. Furthermore, there are enough other buurten available as data input for a multiple regression model to be able to predict the missing buurten well enough for the predictions to be of additional value. Therefore, I conclude that this is possible.